In [1]:
import sys; sys.path.append('../../')  # for correct types inclusion,

import grpc
import pandas as pd

import types_pb2
import types_pb2_grpc
from google.protobuf import timestamp_pb2

import time

SERVER_ADDRESS = 'apiv1.cryptomood.com:443'
PATH_TO_CERT_FILE = './cert.pem'

# THIS CODE WILL TAKE ABOUT 30 SECONDS TO PULL 1 WEEK
def main():
    # Create credentials for use with an secured channel
    credentials = grpc.ssl_channel_credentials(open(PATH_TO_CERT_FILE, 'rb').read())

    # uncomment commands below if token auth is required
    call_credentials = grpc.access_token_call_credentials('ed737d23e01bb1dea60e7e0c16cf3b8a')
    credentials = grpc.composite_channel_credentials(credentials, call_credentials)

    channel = grpc.secure_channel(SERVER_ADDRESS, credentials)

    # create stub
    stub = types_pb2_grpc.SentimentsStub(channel)

    # create timeframe 
    now = time.time()
    seconds = int(now)
    to_time = timestamp_pb2.Timestamp(seconds=seconds)
    from_time = timestamp_pb2.Timestamp(seconds=to_time.seconds - (86400 * 7))  # seven days ago

    # in our case we have to use kwarg because `from` is
    # is recognized as python keyword so there would syntax be error
    # if you want get value you have to use getattr()}
    sentiment_historic_request_kwargs = { 'from': from_time, 'to': to_time, 'resolution': 'M1', 'asset': 'BTC' }
    req = types_pb2.SentimentHistoricRequest(**sentiment_historic_request_kwargs)

    # dataframe inputs
    inputs = []
    index = 0
    df = pd.DataFrame(inputs, columns=['year','month','day','hour','minute','value'])
    candle_stream = stub.HistoricSocialSentiment(req)
    for candle in candle_stream:
        id = candle.id
        value = candle.a
        id = str(id)
        year = id[6:10]
        shortMonth = shortDay = shortHour = False
        adjust = 0

        month = id[18:20]
        if id[18:20].isdigit():
            month = id[18:20]
        else:
            month = id[18:19]
            shortMonth = True;
        if(month == ""):
            month = str(0)

        if shortMonth == False:
            adjust = adjust + 1

        day = id[25+adjust:27+adjust]
        if id[25+adjust:27+adjust].isdigit():
            day = id[25+adjust:27+adjust]
        else:
            day = id[25+adjust:26+adjust]
            shortDay = True;
        if(day == ""):
            day = str(0)

        if shortDay == True:
            adjust = adjust - 1


        hour = id[34+adjust:36+adjust]
        if id[34+adjust:36+adjust].isdigit():
             hour = id[34+adjust:36+adjust]
        else:
            hour = id[34+adjust:35+adjust]
            shortHour = True;
        if(hour == "" or hour == ":" or hour == ": "):
            hour = str(0)

        if shortHour == True:
            adjust = adjust - 1

        if id[45+adjust:47+adjust].isdigit():
             minute = id[45+adjust:47+adjust]
        else:
             minute = id[45+adjust:46+adjust]
             shortMinute = True;
        if(minute == ""):
            minute = str(0)
     
        #print(minute)
        #print(candle.id, candle.a)
        df.loc[index] = [year,month,day,hour,minute,value]
        index = index+1
    
    #print(df)
    df.to_excel("BTC_sentiment.xlsx")

if __name__ == '__main__':
    main()
